In [1]:
pip install --upgrade awscli boto3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install --upgrade s3fs

  Using cached botocore-1.34.69-py3-none-any.whl.metadata (5.7 kB)
  Using cached botocore-1.34.69-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.69-py3-none-any.whl (12.0 MB)
Using cached botocore-1.34.69-py3-none-any.whl (12.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.92
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.92
    Uninstalling botocore-1.34.92:
    Uninstalling botocore-1.34.92:
      Successfully uninstalled botocore-1.34.92
      Successfully uninstalled botocore-1.34.92
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.92 requires botocore==1.34.92, but you have botocore 1.34.69 which is incompatible.
boto3 1.34.92 requires botocore<1.35.0,>=1.34.92, but you have botocore 1.34.69 which is incompatible.
ERROR: pip's dependency resolver does not curr

In [3]:
pip install joblib


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install boto3


  Using cached botocore-1.34.92-py3-none-any.whl.metadata (5.7 kB)
  Using cached botocore-1.34.92-py3-none-any.whl.metadata (5.7 kB)
Using cached botocore-1.34.92-py3-none-any.whl (12.2 MB)
Using cached botocore-1.34.92-py3-none-any.whl (12.2 MB)
  Attempting uninstall: botocore
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.69
    Found existing installation: botocore 1.34.69
    Uninstalling botocore-1.34.69:
    Uninstalling botocore-1.34.69:
      Successfully uninstalled botocore-1.34.69
      Successfully uninstalled botocore-1.34.69
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.12.3 requires botocore<1.34.70,>=1.34.41, but you have botocore 1.34.92 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source 

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from joblib import dump
import tarfile
from joblib import dump



# Załaduj dane
data = pd.read_csv('s3://titanicapi/train.csv')

# Wstępne przetwarzanie
data['Age'].fillna(data['Age'].median(), inplace=True)
data = pd.get_dummies(data, columns=['Sex', 'Embarked', 'Pclass'])

# Podział na cechy i etykiety
X = data.drop(['Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
y = data['Survived']

# Podział na zestawy treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizacja
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model
model = LogisticRegression()
model.fit(X_train, y_train)

# Ocena modelu
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))
# Save the model to a file
model_filename = 'trained_model.joblib'
dump(model, model_filename)
with tarfile.open('trained_model.tar.gz', 'w:gz') as tar:
    tar.add(model_filename)

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

              precision    recall  f1-score   support

           0       0.81      0.86      0.83       105
           1       0.78      0.72      0.75        74

    accuracy                           0.80       179
   macro avg       0.80      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179



/tmp/ipykernel_3037/1710510363.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)
/tmp/ipykernel_3037/1710510363.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [6]:
import boto3

def upload_file_to_s3(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified, file_name is used
    """
    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except Exception as e:
        print(f'Error uploading file: {e}')
        return False
    return True

# Call the function
upload_successful = upload_file_to_s3('trained_model.tar.gz', 'titanicapi')
if upload_successful:
    print("Model uploaded successfully!")
else:
    print("Failed to upload the model.")


Model uploaded successfully!
Model uploaded successfully!


In [7]:
import sagemaker
from sagemaker.sklearn.model import SKLearnModel

session = sagemaker.Session()

model = SKLearnModel(model_data='s3://titanicapi/trained_model.tar.gz',
                     role=sagemaker.get_execution_role(),
                     entry_point='inference.py',  # This script should contain functions for model loading and inference
                     framework_version='0.23-1',
                     sagemaker_session=session)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [8]:
predictor = model.deploy(initial_instance_count=1,
                         instance_type='ml.t2.medium')

------------------------------------------------------------------------------------------------------------**

UnexpectedStatusException: Error hosting endpoint sagemaker-scikit-learn-2024-04-25-20-04-26-178: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html

UnexpectedStatusException: Error hosting endpoint sagemaker-scikit-learn-2024-04-25-20-04-26-178: Failed. Reason: The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint.. Try changing the instance type or reference the troubleshooting page https://docs.aws.amazon.com/sagemaker/latest/dg/async-inference-troubleshooting.html